In [16]:
from multiprocessing import Pool, cpu_count
import gc; gc.enable()
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn import *
import sklearn
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression

In [77]:
# reading wsdm csv
wsdm_data = pd.read_csv(r"D:\Work_Masterarbeit\wsdm\train_v2.csv")

# reading members csv
members_data = pd.read_csv(r"D:\Work_Masterarbeit\wsdm\members_v3.csv")

# merging train dataset with members dataset
wsdm_members = pd.merge(wsdm_data, members_data, on='msno', how='left')
# getting the head (top 5 rows) of df
wsdm_members.head()

,msno,is_churn,city,bd,gender,registered_via,registration_init_time
0,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1,5.0,28.0,male,3.0,20131223.0
1,f/NmvEzHfhINFEYZTR05prUdr+E+3+oewvweYz9cCQE=,1,13.0,20.0,male,3.0,20131223.0
2,zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,1,13.0,18.0,male,3.0,20131227.0
3,8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=,1,1.0,0.0,NaN,7.0,20140109.0
4,K6fja4+jmoZ5xG6BypqX80Uw/XKpMgrEMdG2edFOxnA=,1,13.0,35.0,female,7.0,20140125.0


In [78]:
### 4.1 Loading files to merge-

In [79]:
# Laden der Daten wsdm Daten [VOV]
wsdm_data = pd.read_csv(r"D:\Work_Masterarbeit\wsdm\train_v2.csv")
# members data
members_data = pd.read_csv(r"D:\Work_Masterarbeit\wsdm\members_v3.csv")
# transactions data
transactions_data = pd.read_csv(r"D:\Work_Masterarbeit\wsdm\transactions_v2.csv")
# user logs data
logs_data = pd.read_csv(r"D:\Work_Masterarbeit\wsdm\user_logs_v2.csv")

In [80]:
### 4.2 Merging files together-
### 4.2.1 For train data-

In [81]:
# Zusammenführen der Daten [VOV]
wsdm_members = pd.merge(wsdm_data, members_data, on='msno', how='left')
# merging transactions data with train members data
wsdm_mem_trans = pd.merge(wsdm_members, transactions_data, on='msno', how='left')
# merging user logs data with train members transaction data
wsdm_dataset = pd.merge(wsdm_mem_trans, logs_data, on='msno', how='left')
# getting head of train dataset
pd.set_option('display.max_columns', 100)
wsdm_dataset.head()

,msno,is_churn,city,bd,gender,registered_via,registration_init_time,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
0,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1,5.0,28.0,male,3.0,20131223.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20170305.0,7.0,0.0,3.0,5.0,71.0,68.0,17599.893
1,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1,5.0,28.0,male,3.0,20131223.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20170301.0,138.0,19.0,7.0,1.0,21.0,158.0,8830.433
2,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1,5.0,28.0,male,3.0,20131223.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20170319.0,0.0,0.0,0.0,0.0,34.0,17.0,7883.313
3,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1,5.0,28.0,male,3.0,20131223.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20170316.0,15.0,0.0,0.0,1.0,38.0,17.0,9029.227
4,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1,5.0,28.0,male,3.0,20131223.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20170310.0,0.0,0.0,0.0,0.0,8.0,8.0,1870.110


In [82]:
wsdm_dataset.shape

(16887877, 23)

In [83]:
# [VOV] since registration_init_time contains a lot of nan values so impute median date
wsdm_dataset['registration_init_time'] = wsdm_dataset['registration_init_time'].fillna(20150204.0)
# getting registration_init_time array
reg_date = wsdm_dataset['registration_init_time'].values
reg_date

array([20131223., 20131223., 20131223., ..., 20110905., 20110905.,
       20150204.])

In [84]:
# storing these train registrationn dates to npz file
np.savez_compressed(r"D:\Work_Masterarbeit\wsdm\preprocessed\wsdm_registration_date", reg_date)

<h3> Vorverarbeitung
   

In [85]:
def preprocess(data):
    '''[VOV] Diese Vorverarbeitungsfunktion wird verwendet, um grundlegende Vorverarbeitungen am Anfang des 
    Trainings- und Testdatensatzes durchzuführen. Diese Vorverarbeitung besteht aus der Entfernung von Ausreißern, 
    der Imputation von Null Values und dem Ersetzen der Werte'''
    
    # imputing 0 in place of nan values in the city column
    data['city'] = data['city'].fillna(0)

    # removing outliers
    data['bd'] = data['bd'].apply(lambda x: x if (x < 72.0) and (x > 0.0) else np.nan)
    # imputing 28 as age instead of nan
    data['bd'] = data['bd'].fillna(28.0)

    # replacing male with 1 in gender
    data['gender'] = data['gender'].replace(to_replace='male', value=1)
    # replacing male with 2 in gender
    data['gender'] = data['gender'].replace(to_replace='female', value=2)
    # replacing nan with 0 in gender
    data['gender'] = data['gender'].fillna(0)

    # replace 0 instead of nan in registered_via
    data['registered_via'] = data['registered_via'].fillna(0)

    # filling median date in place of nan in the df
    data['registration_init_time'] = data['registration_init_time'].fillna(20150204.0)
    # converting float date to datetime
    data['registration_init_time'] = pd.to_datetime(data['registration_init_time'], format='%Y%m%d')

    # imputing 0 in place of nan value in payment_method_id
    data['payment_method_id'] = data['payment_method_id'].fillna(0)

    # removing outliers
    data['payment_plan_days'] = data['payment_plan_days'].apply(lambda x: x if (x <= 30.0) else np.nan)
    # imputing 30 in place of nan in payment_plan_days
    data['payment_plan_days'] = data['payment_plan_days'].fillna(30.0)

    # removing outliers
    data['plan_list_price'] = data['plan_list_price'].apply(lambda x: x if (x <= 180.0) else np.nan)
    # imputing 149 in place of nan in plan_list_price
    data['plan_list_price'] = data['plan_list_price'].fillna(149.0)

    # removing outliers
    data['actual_amount_paid'] = data['actual_amount_paid'].apply(lambda x: x if (x <= 180.0) else np.nan)
    # imputing 149 in place of nan in actual_amount_paid
    data['actual_amount_paid'] = data['actual_amount_paid'].fillna(149.0)

    # imputing 2 in place of nan values in is_auto_renew
    data['is_auto_renew'] = data['is_auto_renew'].fillna(2)

    # filling median date in place of nan in the df
    data['transaction_date'] = data['transaction_date'].fillna(20170316.0)
    # converting float date to datetime
    data['transaction_date'] = pd.to_datetime(data['transaction_date'], format='%Y%m%d')

    # filling median date in place of nan in the df
    data['membership_expire_date'] = data['membership_expire_date'].fillna(20170419.0)
    # converting float date to datetime
    data['membership_expire_date'] = pd.to_datetime(data['membership_expire_date'], format='%Y%m%d')

    # imputing 2 in place of nan values in is_cancel
    data['is_cancel'] = data['is_cancel'].fillna(2)

    # filling median date in place of nan in the df
    data['date'] = data['date'].fillna(20170316.0)
    # converting float date to datetime
    data['date'] = pd.to_datetime(data['date'], format='%Y%m%d')
    
    # removing outliers
    data['num_25'] = data['num_25'].apply(lambda x: x if (x <= 15.0) else np.nan)
    # now I can impute 2 instead of nan in num_25
    data['num_25'] = data['num_25'].fillna(2.0)

    # removing outliers
    data['num_50'] = data['num_50'].apply(lambda x: x if (x <= 4.0) else np.nan)
    # now I can impute 0 instead of nan in num_50
    data['num_50'] = data['num_50'].fillna(0)

    # removing outliers
    data['num_75'] = data['num_75'].apply(lambda x: x if (x <= 3.0) else np.nan)
    # now I can impute 0 instead of nan in num_75
    data['num_75'] = data['num_75'].fillna(0)

    # removing outliers
    data['num_985'] = data['num_985'].apply(lambda x: x if (x <= 3.0) else np.nan)
    # now I can impute 0 instead of nan in num_985
    data['num_985'] = data['num_985'].fillna(0)

    # removing outliers
    data['num_100'] = data['num_100'].apply(lambda x: x if (x <= 74.0) else np.nan)
    # now I can impute 14 instead of nan in num_100
    data['num_100'] = data['num_100'].fillna(14.0)

    # removing outliers
    data['num_unq'] = data['num_unq'].apply(lambda x: x if (x <= 68.0) else np.nan)
    # now I can impute 16 instead of nan in num_unq
    data['num_unq'] = data['num_unq'].fillna(16.0)

    # removing outliers
    data['total_secs'] = data['total_secs'].apply(lambda x: x if (x <= 19167.549700000025) else np.nan)
    # now I can impute 3880.765 instead of nan in total_secs
    data['total_secs'] = data['total_secs'].fillna(3880.765)

In [86]:
# Checking null values for train data
# getting features wise null values (number)
wsdm_dataset.isnull().sum()

msno                            0
is_churn                        0
city                       117166
bd                         117166
gender                    7860346
registered_via             117166
registration_init_time          0
payment_method_id          340641
payment_plan_days          340641
plan_list_price            340641
actual_amount_paid         340641
is_auto_renew              340641
transaction_date           340641
membership_expire_date     340641
is_cancel                  340641
date                       249615
num_25                     249615
num_50                     249615
num_75                     249615
num_985                    249615
num_100                    249615
num_unq                    249615
total_secs                 249615
dtype: int64

In [87]:
# performing preprocessing on top of entire train data
preprocess(wsdm_dataset)

In [88]:
# again just to cross check if there is any null or not
wsdm_dataset.isnull().values.any()

False

In [89]:
# checking for any duplicate row
wsdm_dataset.duplicated().any()

False

In [90]:
# saving train file to disk
wsdm_dataset.to_csv(r"D:\Work_Masterarbeit\wsdm\preprocessed\train.csv", index=False)

<h2>Feature Engineering

In [91]:
# loading the files

wsdm_dataset = pd.read_csv(r"D:\Work_Masterarbeit\wsdm\preprocessed\train.csv")

In [92]:
# getting head of wsdm file
pd.set_option('display.max_columns', 100)
wsdm_dataset.head()

,msno,is_churn,city,bd,gender,registered_via,registration_init_time,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
0,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1,5.0,28.0,1.0,3.0,2013-12-23,0.0,30.0,149.0,149.0,2.0,2017-03-16,2017-04-19,2.0,2017-03-05,7.0,0.0,3.0,0.0,71.0,68.0,17599.893
1,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1,5.0,28.0,1.0,3.0,2013-12-23,0.0,30.0,149.0,149.0,2.0,2017-03-16,2017-04-19,2.0,2017-03-01,2.0,0.0,0.0,1.0,21.0,16.0,8830.433
2,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1,5.0,28.0,1.0,3.0,2013-12-23,0.0,30.0,149.0,149.0,2.0,2017-03-16,2017-04-19,2.0,2017-03-19,0.0,0.0,0.0,0.0,34.0,17.0,7883.313
3,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1,5.0,28.0,1.0,3.0,2013-12-23,0.0,30.0,149.0,149.0,2.0,2017-03-16,2017-04-19,2.0,2017-03-16,15.0,0.0,0.0,1.0,38.0,17.0,9029.227
4,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1,5.0,28.0,1.0,3.0,2013-12-23,0.0,30.0,149.0,149.0,2.0,2017-03-16,2017-04-19,2.0,2017-03-10,0.0,0.0,0.0,0.0,8.0,8.0,1870.110


<h2> Hier Baseline für ADL

In [93]:
# Feature - 1 (getting weekday from date)

# getting weekday from the date feature for train data
wsdm_dataset['date'] = pd.to_datetime(wsdm_dataset['date'], errors='coerce')
wsdm_dataset['day_of_the_week'] = wsdm_dataset['date'].dt.day_name().values


In [94]:
# Feature - 2 (checking for the weekend)

wsdm_dataset['is_weekend'] = wsdm_dataset['day_of_the_week'].apply(lambda x: 1 if (x == 'Saturday') or (x == 'Sunday') else 0)

In [95]:
# Feature - 3 (checking for the weekday)

wsdm_dataset['is_weekday'] = wsdm_dataset['is_weekend'].apply(lambda x: 1 if (x == 0) else 0)


In [96]:
%%time
# Feature - 4 to 12 (sum based features)
# Feature - 13 to 19 (mean based features)
# Feature - 20 to 27 (standard deviation based features)
# Feature - 28 (nunique based feature)
# Feature - 29 and 30 (min and max based features)
# Feature - 31 to 33 (mean based features for transaction)
# Feature - 34 (transaction count)
# Feature - 35 (transaction date max)
# Feature - 36 (membership expiry date max)
# Feature - 37 (membership expiry date count)

def std(x):
    '''finding standard deviation using numpy,
    to avoid getting nan values'''
    return np.std(x)

# grouping them together for train data
temp_df_train = wsdm_dataset.groupby('msno').agg(num_25_sum=('num_25', 'sum'),
                                num_50_sum=('num_50', 'sum'),
                                num_75_sum=('num_75', 'sum'),
                                num_985_sum=('num_985', 'sum'),
                                num_100_sum=('num_100', 'sum'),
                                num_unq_sum=('num_unq', 'sum'),
                                total_secs_sum=('total_secs', 'sum'),
                                is_weekend_sum=('is_weekend', 'sum'),
                                is_weekday_sum=('is_weekday', 'sum'),
                                num_25_mean=('num_25', 'mean'),
                                num_50_mean=('num_50', 'mean'),
                                num_75_mean=('num_75', 'mean'),
                                num_985_mean=('num_985', 'mean'),
                                num_100_mean=('num_100', 'mean'),
                                num_unq_mean=('num_unq', 'mean'),
                                total_secs_mean=('total_secs', 'mean'),
                                num_25_std=('num_25', std),
                                num_50_std=('num_50', std),
                                num_75_std=('num_75', std),
                                num_985_std=('num_985', std),
                                num_100_std=('num_100', std),
                                num_unq_std=('num_unq', std),
                                total_secs_std=('total_secs', std),
                                active_days=('date', 'nunique'),
                                date_min=('date', 'min'),
                                date_max=('date', 'max'),
                                payment_plan_days_mean=('payment_plan_days', 'mean'),
                                plan_list_price_mean=('plan_list_price', 'mean'),
                                actual_amount_paid_mean=('actual_amount_paid', 'mean'),
                                transaction_date_count=('transaction_date', 'nunique'),
                                transaction_date_max=('transaction_date', 'max'),
                                membership_expire_date_max=('membership_expire_date', 'max'),
                                membership_expire_count=('membership_expire_date', 'nunique'))
# merging them with the train dataset
wsdm_dataset = pd.merge(wsdm_dataset, temp_df_train, on='msno', how='left')

Wall time: 5min 9s


In [97]:
# Feature - 38 (activity period)

wsdm_dataset['date_min'] = pd.to_datetime(wsdm_dataset['date_min'], errors='coerce')
wsdm_dataset['date_max'] = pd.to_datetime(wsdm_dataset['date_max'], errors='coerce')


wsdm_dataset['activity_period'] = (wsdm_dataset['date_max'] - wsdm_dataset['date_min']).dt.days + 1

In [98]:
# Feature - 39 (inactive days)

wsdm_dataset['inactive_days'] = wsdm_dataset['date'].nunique() - wsdm_dataset['active_days']

In [99]:
# Feature - 40 (rare behaviour)
# 0 (for not rare user) and 1 (for rare user)

wsdm_dataset['is_rare'] = wsdm_dataset['active_days'].apply(lambda x: 0 if (x > 1) else 1)

In [100]:
# Feature - 41 (average time per day)

wsdm_dataset['avg_time_perday'] = wsdm_dataset['total_secs_sum'] / wsdm_dataset['active_days']


# Feature - 42 (unique tracks played per day)

wsdm_dataset['unq_track_perday'] = round(wsdm_dataset['num_unq_sum'] / wsdm_dataset['active_days'])


# Feature - 43 (tracks played till 25% length per day)

wsdm_dataset['till_25_perday'] = round(wsdm_dataset['num_25_sum'] / wsdm_dataset['active_days'])


# Feature - 44 (tracks played till 50% length per day)

wsdm_dataset['till_50_perday'] = round(wsdm_dataset['num_50_sum'] / wsdm_dataset['active_days'])


# Feature - 45 (tracks played till 75% length per day)

wsdm_dataset['till_75_perday'] = round(wsdm_dataset['num_75_sum'] / wsdm_dataset['active_days'])


# Feature - 46 (tracks played till 98.5% length per day)

wsdm_dataset['till_985_perday'] = round(wsdm_dataset['num_985_sum'] / wsdm_dataset['active_days'])


# Feature - 47 (tracks played till 100% length per day)

wsdm_dataset['till_full_perday'] = round(wsdm_dataset['num_100_sum'] / wsdm_dataset['active_days'])

In [101]:
# Feature - 48 (discount)
wsdm_dataset['discount'] = wsdm_dataset['plan_list_price'] - wsdm_dataset['actual_amount_paid']

In [102]:
wsdm_dataset['discount'].unique()

array([   0.,  149.,  180., -149.,   30., -119., -129.,  120.,   20.,
         50., -100.,   -1.,   99.,  129.,    1.])

In [103]:
# since there are several values which are less than 0 for various reasons
# so it's always good to set a lower limit, here I can set 0
wsdm_dataset['discount'] = wsdm_dataset['discount'].clip(lower=0)

In [104]:
# Feature - 49 (is_discount)
wsdm_dataset['is_discount'] = wsdm_dataset['discount'].apply(lambda x: 1 if (x > 0) else 0)

In [105]:
# Feature - 50 (days since final login)
# Threshold that I set here is the last date, which is 31-03-2017

wsdm_dataset['days_since_last_login'] = (pd.to_datetime(wsdm_dataset['date'].unique().max(), errors='coerce') - wsdm_dataset['date_max']).dt.days

In [106]:
wsdm_dataset['membership_expire_date_max'] = pd.to_datetime(wsdm_dataset['membership_expire_date_max'], errors='coerce')

wsdm_dataset['days_left'] = (wsdm_dataset['membership_expire_date_max'] - pd.to_datetime(wsdm_dataset['date'].unique().max(), errors='coerce')).dt.days

In [107]:
# Since there are many negative values so I can set them to 0
wsdm_dataset['days_left'] = wsdm_dataset['days_left'].clip(lower=0)

In [108]:
# Feature - 51 (Loyality range)

wsdm_dataset['transaction_date_max'] = pd.to_datetime(wsdm_dataset['transaction_date_max'], errors='coerce')
wsdm_dataset['registration_init_time'] = pd.to_datetime(wsdm_dataset['registration_init_time'], errors='coerce')

wsdm_dataset['layality_range'] = (wsdm_dataset['transaction_date_max'] - wsdm_dataset['registration_init_time']).dt.days

In [109]:
# Feature - 52 (price per day)

wsdm_dataset['Perday_price'] = wsdm_dataset['actual_amount_paid'] / wsdm_dataset['payment_plan_days']

In [110]:
# Feature - 53 (days since final transaction)

wsdm_dataset['days_since_final_transaction'] = (pd.to_datetime(wsdm_dataset['date'].unique().max(), errors='coerce') - wsdm_dataset['transaction_date_max']).dt.days

In [111]:
# all features in train data
wsdm_dataset.columns

Index(['msno', 'is_churn', 'city', 'bd', 'gender', 'registered_via',
       'registration_init_time', 'payment_method_id', 'payment_plan_days',
       'plan_list_price', 'actual_amount_paid', 'is_auto_renew',
       'transaction_date', 'membership_expire_date', 'is_cancel', 'date',
       'num_25', 'num_50', 'num_75', 'num_985', 'num_100', 'num_unq',
       'total_secs', 'day_of_the_week', 'is_weekend', 'is_weekday',
       'num_25_sum', 'num_50_sum', 'num_75_sum', 'num_985_sum', 'num_100_sum',
       'num_unq_sum', 'total_secs_sum', 'is_weekend_sum', 'is_weekday_sum',
       'num_25_mean', 'num_50_mean', 'num_75_mean', 'num_985_mean',
       'num_100_mean', 'num_unq_mean', 'total_secs_mean', 'num_25_std',
       'num_50_std', 'num_75_std', 'num_985_std', 'num_100_std', 'num_unq_std',
       'total_secs_std', 'active_days', 'date_min', 'date_max',
       'payment_plan_days_mean', 'plan_list_price_mean',
       'actual_amount_paid_mean', 'transaction_date_count',
       'transaction_dat

In [112]:
# reference: https://medium.com/@vincentteyssier/optimizing-the-size-of-a-pandas-dataframe-for-low-memory-environment-5f07db3d72e
# reference: https://numpy.org/doc/stable/reference/generated/numpy.iinfo.html
# reference: https://numpy.org/doc/stable/reference/generated/numpy.finfo.html

def datatype_changer(dataset):
    # iterating through all the columns in the dataframe
    for col in dataset.columns:
        # getting column's datatype
        col_type = dataset[col].dtype
        
        # checking if datatype of column is 'object' or not
        # if column type is not object
        if (col_type == int) or (col_type == float):
            # getting minimum value of a column
            min_val = dataset[col].min()
            # getting maximum value of a column
            max_val = dataset[col].max()
            # checking whether the datatype contain first 3 characters as int or not, if int
            if str(col_type)[:3] == 'int':
                # cheking the minimal and maximal value for int8, int16, int32 and int64 in numpy
                if min_val > np.iinfo(np.int8).min and max_val < np.iinfo(np.int8).max:
                    dataset[col] = dataset[col].astype(np.int8)
                elif min_val > np.iinfo(np.int16).min and max_val < np.iinfo(np.int16).max:
                    dataset[col] = dataset[col].astype(np.int16)
                elif min_val > np.iinfo(np.int32).min and max_val < np.iinfo(np.int32).max:
                    dataset[col] = dataset[col].astype(np.int32)
                else:
                    dataset[col] = dataset[col].astype(np.int64)
            else:
                # if it is non int, which is ultimately float
                # cheking the minimal and maximal value for float16, float32 and float64 in numpy
                if min_val > np.finfo(np.float16).min and max_val < np.finfo(np.float16).max:
                    dataset[col] = dataset[col].astype(np.float16)
                elif min_val > np.finfo(np.float32).min and max_val < np.finfo(np.float32).max:
                    dataset[col] = dataset[col].astype(np.float32)
                else:
                    dataset[col] = dataset[col].astype(np.float64)
        else:
            # keeping rest of them to category datatype instead of object
            dataset[col] = dataset[col].astype('category')
            
    # returning head of the dataframe
    return dataset

In [113]:
wsdm_dataset = datatype_changer(wsdm_dataset)

In [115]:
# saving train file

wsdm_dataset.to_csv(r"D:\Work_Masterarbeit\wsdm\preprocessed\wsdm_FA.csv", index=False)

<h1>3. Prepare Data for Modelling

In [2]:
# loading train dataset
wsdm_dataset = pd.read_csv(r"D:\Work_Masterarbeit\wsdm\preprocessed\wsdm_FA.csv")
# setting max_columns to 100, so it displays all the columns of train dataset
pd.set_option('display.max_columns', 100)
# displaying head (top 5) rows of train dataset
wsdm_dataset.head()

,msno,is_churn,city,bd,gender,registered_via,registration_init_time,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs,day_of_the_week,is_weekend,is_weekday,num_25_sum,num_50_sum,num_75_sum,num_985_sum,num_100_sum,num_unq_sum,total_secs_sum,is_weekend_sum,is_weekday_sum,num_25_mean,num_50_mean,num_75_mean,num_985_mean,num_100_mean,num_unq_mean,total_secs_mean,num_25_std,num_50_std,num_75_std,num_985_std,num_100_std,num_unq_std,total_secs_std,active_days,date_min,date_max,payment_plan_days_mean,plan_list_price_mean,actual_amount_paid_mean,transaction_date_count,transaction_date_max,membership_expire_date_max,membership_expire_count,activity_period,inactive_days,is_rare,avg_time_perday,unq_track_perday,till_25_perday,till_50_perday,till_75_perday,till_985_perday,till_full_perday,discount,is_discount,days_since_last_login,days_left,layality_range,Perday_price,days_since_final_transaction
0,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1,5.0,28.0,1.0,3.0,1387756800000000000,0.0,30.0,149.0,149.0,2.0,2017-03-16,2017-04-19,2.0,1488672000000000000,7.0,0.0,3.0,0.0,71.0,68.0,17600.0,Sunday,1,0,50.0,4.0,6.0,5.0,246.0,206.0,64499.977,5,6,4.547,0.3635,0.5454,0.4546,22.36,18.73,5864.0,4.734,0.481,0.8906,0.8906,19.42,16.14,4748.0,11,1488326400000000000,1490745600000000000,30.0,149.0,149.0,1,1489622400000000000,1492560000000000000,1,29,20,0,5863.6343,19.0,5.0,0.0,1.0,0.0,22.0,0.0,0,2,19,1179,4.966667,15
1,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1,5.0,28.0,1.0,3.0,1387756800000000000,0.0,30.0,149.0,149.0,2.0,2017-03-16,2017-04-19,2.0,1488326400000000000,2.0,0.0,0.0,1.0,21.0,16.0,8830.0,Wednesday,0,1,50.0,4.0,6.0,5.0,246.0,206.0,64499.977,5,6,4.547,0.3635,0.5454,0.4546,22.36,18.73,5864.0,4.734,0.481,0.8906,0.8906,19.42,16.14,4748.0,11,1488326400000000000,1490745600000000000,30.0,149.0,149.0,1,1489622400000000000,1492560000000000000,1,29,20,0,5863.6343,19.0,5.0,0.0,1.0,0.0,22.0,0.0,0,2,19,1179,4.966667,15
2,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1,5.0,28.0,1.0,3.0,1387756800000000000,0.0,30.0,149.0,149.0,2.0,2017-03-16,2017-04-19,2.0,1489881600000000000,0.0,0.0,0.0,0.0,34.0,17.0,7884.0,Sunday,1,0,50.0,4.0,6.0,5.0,246.0,206.0,64499.977,5,6,4.547,0.3635,0.5454,0.4546,22.36,18.73,5864.0,4.734,0.481,0.8906,0.8906,19.42,16.14,4748.0,11,1488326400000000000,1490745600000000000,30.0,149.0,149.0,1,1489622400000000000,1492560000000000000,1,29,20,0,5863.6343,19.0,5.0,0.0,1.0,0.0,22.0,0.0,0,2,19,1179,4.966667,15
3,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1,5.0,28.0,1.0,3.0,1387756800000000000,0.0,30.0,149.0,149.0,2.0,2017-03-16,2017-04-19,2.0,1489622400000000000,15.0,0.0,0.0,1.0,38.0,17.0,9030.0,Thursday,0,1,50.0,4.0,6.0,5.0,246.0,206.0,64499.977,5,6,4.547,0.3635,0.5454,0.4546,22.36,18.73,5864.0,4.734,0.481,0.8906,0.8906,19.42,16.14,4748.0,11,1488326400000000000,1490745600000000000,30.0,149.0,149.0,1,1489622400000000000,1492560000000000000,1,29,20,0,5863.6343,19.0,5.0,0.0,1.0,0.0,22.0,0.0,0,2,19,1179,4.966667,15
4,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1,5.0,28.0,1.0,3.0,1387756800000000000,0.0,30.0,149.0,149.0,2.0,2017-03-16,2017-04-19,2.0,1489104000000000000,0.0,0.0,0.0,0.0,8.0,8.0,1870.0,Friday,0,1,50.0,4.0,6.0,5.0,246.0,206.0,64499.977,5,6,4.547,0.3635,0.5454,0.4546,22.36,18.73,5864.0,4.734,0.481,0.8906,0.8906,19.42,16.14,4748.0,11,1488326400000000000,1490745600000000000,30.0,149.0,149.0,1,1489622400000000000,1492560000000000000,1,29,20,0,5863.6343,19.0,5.0,0.0,1.0,0.0,22.0,0.0,0,2,19,1179,4.966667,15


In [3]:
# removing all non_useful features first
wsdm_dataset = wsdm_dataset.drop(['registration_init_time',
                              'transaction_date', 
                              'membership_expire_date', 
                              'date', 
                              'num_25',
                              'num_50',
                              'num_75',
                              'num_985',
                              'num_100',
                              'num_unq',
                              'total_secs',
                              'is_weekend',
                              'is_weekday',
                              'day_of_the_week', 
                              'date_min', 
                              'date_max', 
                              'transaction_date_max', 
                              'membership_expire_date_max'], axis=1, inplace=False)

In [4]:
# getting shape of train dataset
wsdm_dataset.shape

(16887877, 58)

Now this train dataset has 16887877 rows and 58 columns

In [5]:
# adding registration init time feature to train dataset
# load the registration array
reg_date = np.load(r"D:\Work_Masterarbeit\wsdm\preprocessed\wsdm_registration_date.npz")
# display the array
reg_date['arr_0']

array([20131223., 20131223., 20131223., ..., 20110905., 20110905.,
       20150204.])

In [6]:
# converting numpy array to list
registration_init_time = reg_date['arr_0'].tolist()
# getting length of the list
len(registration_init_time)

16887877

In [7]:
# creating new feature, and adding it to train dataset
wsdm_dataset['registration_init_time'] = registration_init_time

In [8]:
# setting max_columns to 100, so it displays all the columns of train dataset
pd.set_option('display.max_columns', 100)
# displaying head (top 5) rows of train dataset
wsdm_dataset.head()

,msno,is_churn,city,bd,gender,registered_via,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,is_cancel,num_25_sum,num_50_sum,num_75_sum,num_985_sum,num_100_sum,num_unq_sum,total_secs_sum,is_weekend_sum,is_weekday_sum,num_25_mean,num_50_mean,num_75_mean,num_985_mean,num_100_mean,num_unq_mean,total_secs_mean,num_25_std,num_50_std,num_75_std,num_985_std,num_100_std,num_unq_std,total_secs_std,active_days,payment_plan_days_mean,plan_list_price_mean,actual_amount_paid_mean,transaction_date_count,membership_expire_count,activity_period,inactive_days,is_rare,avg_time_perday,unq_track_perday,till_25_perday,till_50_perday,till_75_perday,till_985_perday,till_full_perday,discount,is_discount,days_since_last_login,days_left,layality_range,Perday_price,days_since_final_transaction,registration_init_time
0,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1,5.0,28.0,1.0,3.0,0.0,30.0,149.0,149.0,2.0,2.0,50.0,4.0,6.0,5.0,246.0,206.0,64499.977,5,6,4.547,0.3635,0.5454,0.4546,22.36,18.73,5864.0,4.734,0.481,0.8906,0.8906,19.42,16.14,4748.0,11,30.0,149.0,149.0,1,1,29,20,0,5863.6343,19.0,5.0,0.0,1.0,0.0,22.0,0.0,0,2,19,1179,4.966667,15,20131223.0
1,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1,5.0,28.0,1.0,3.0,0.0,30.0,149.0,149.0,2.0,2.0,50.0,4.0,6.0,5.0,246.0,206.0,64499.977,5,6,4.547,0.3635,0.5454,0.4546,22.36,18.73,5864.0,4.734,0.481,0.8906,0.8906,19.42,16.14,4748.0,11,30.0,149.0,149.0,1,1,29,20,0,5863.6343,19.0,5.0,0.0,1.0,0.0,22.0,0.0,0,2,19,1179,4.966667,15,20131223.0
2,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1,5.0,28.0,1.0,3.0,0.0,30.0,149.0,149.0,2.0,2.0,50.0,4.0,6.0,5.0,246.0,206.0,64499.977,5,6,4.547,0.3635,0.5454,0.4546,22.36,18.73,5864.0,4.734,0.481,0.8906,0.8906,19.42,16.14,4748.0,11,30.0,149.0,149.0,1,1,29,20,0,5863.6343,19.0,5.0,0.0,1.0,0.0,22.0,0.0,0,2,19,1179,4.966667,15,20131223.0
3,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1,5.0,28.0,1.0,3.0,0.0,30.0,149.0,149.0,2.0,2.0,50.0,4.0,6.0,5.0,246.0,206.0,64499.977,5,6,4.547,0.3635,0.5454,0.4546,22.36,18.73,5864.0,4.734,0.481,0.8906,0.8906,19.42,16.14,4748.0,11,30.0,149.0,149.0,1,1,29,20,0,5863.6343,19.0,5.0,0.0,1.0,0.0,22.0,0.0,0,2,19,1179,4.966667,15,20131223.0
4,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1,5.0,28.0,1.0,3.0,0.0,30.0,149.0,149.0,2.0,2.0,50.0,4.0,6.0,5.0,246.0,206.0,64499.977,5,6,4.547,0.3635,0.5454,0.4546,22.36,18.73,5864.0,4.734,0.481,0.8906,0.8906,19.42,16.14,4748.0,11,30.0,149.0,149.0,1,1,29,20,0,5863.6343,19.0,5.0,0.0,1.0,0.0,22.0,0.0,0,2,19,1179,4.966667,15,20131223.0


In [9]:
# saving new dataset
wsdm_dataset.to_csv(r"D:\Work_Masterarbeit\wsdm\preprocessed\prepared_train.csv", index=False)

In [10]:
# reading csv file for removing duplicates
wsdm_dataset = pd.read_csv(r"D:\Work_Masterarbeit\wsdm\preprocessed\prepared_train.csv")
# getting all columns of train dataset
wsdm_dataset.columns

Index(['msno', 'is_churn', 'city', 'bd', 'gender', 'registered_via',
       'payment_method_id', 'payment_plan_days', 'plan_list_price',
       'actual_amount_paid', 'is_auto_renew', 'is_cancel', 'num_25_sum',
       'num_50_sum', 'num_75_sum', 'num_985_sum', 'num_100_sum', 'num_unq_sum',
       'total_secs_sum', 'is_weekend_sum', 'is_weekday_sum', 'num_25_mean',
       'num_50_mean', 'num_75_mean', 'num_985_mean', 'num_100_mean',
       'num_unq_mean', 'total_secs_mean', 'num_25_std', 'num_50_std',
       'num_75_std', 'num_985_std', 'num_100_std', 'num_unq_std',
       'total_secs_std', 'active_days', 'payment_plan_days_mean',
       'plan_list_price_mean', 'actual_amount_paid_mean',
       'transaction_date_count', 'membership_expire_count', 'activity_period',
       'inactive_days', 'is_rare', 'avg_time_perday', 'unq_track_perday',
       'till_25_perday', 'till_50_perday', 'till_75_perday', 'till_985_perday',
       'till_full_perday', 'discount', 'is_discount', 'days_since_last_l

In [11]:
# removing all duplicate rows for a particular id
wsdm_dataset = wsdm_dataset.drop_duplicates('msno', keep='first', inplace=False)
# setting max_columns to 100, so it displays all the columns of train dataset
pd.set_option('display.max_columns', 100)
# displaying head (top 10) rows of train dataset
wsdm_dataset.head(10)

,msno,is_churn,city,bd,gender,registered_via,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,is_cancel,num_25_sum,num_50_sum,num_75_sum,num_985_sum,num_100_sum,num_unq_sum,total_secs_sum,is_weekend_sum,is_weekday_sum,num_25_mean,num_50_mean,num_75_mean,num_985_mean,num_100_mean,num_unq_mean,total_secs_mean,num_25_std,num_50_std,num_75_std,num_985_std,num_100_std,num_unq_std,total_secs_std,active_days,payment_plan_days_mean,plan_list_price_mean,actual_amount_paid_mean,transaction_date_count,membership_expire_count,activity_period,inactive_days,is_rare,avg_time_perday,unq_track_perday,till_25_perday,till_50_perday,till_75_perday,till_985_perday,till_full_perday,discount,is_discount,days_since_last_login,days_left,layality_range,Perday_price,days_since_final_transaction,registration_init_time
0,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1,5.0,28.0,1.0,3.0,0.0,30.0,149.0,149.0,2.0,2.0,50.0,4.0,6.0,5.0,246.0,206.0,64499.977,5,6,4.547,0.3635,0.5454,0.4546,22.360,18.73,5864.0,4.734,0.4810,0.8906,0.8906,19.420,16.140,4748.0,11,30.0,149.0,149.0,1,1,29,20,0,5863.6343,19.0,5.0,0.0,1.0,0.0,22.0,0.0,0,2,19,1179,4.966667,15,20131223.0
11,f/NmvEzHfhINFEYZTR05prUdr+E+3+oewvweYz9cCQE=,1,13.0,20.0,1.0,3.0,36.0,30.0,180.0,180.0,0.0,0.0,0.0,4.0,2.0,0.0,26.0,30.0,6986.509,3,3,0.000,0.6665,0.3333,0.0000,4.332,5.00,1164.0,0.000,1.1050,0.4714,0.0000,2.926,2.887,741.5,6,30.0,180.0,180.0,1,1,20,25,0,1164.4182,5.0,0.0,1.0,0.0,0.0,4.0,0.0,0,11,11,1174,6.000000,20,20131223.0
17,zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,1,13.0,18.0,1.0,3.0,17.0,30.0,0.0,0.0,0.0,0.0,150.0,50.0,38.0,30.0,410.0,732.0,135620.940,16,24,3.750,1.2500,0.9500,0.7500,10.250,18.30,3390.0,2.980,1.4795,1.0710,0.7666,12.900,19.250,3882.0,20,30.0,74.5,74.5,2,2,31,11,0,6781.0470,37.0,8.0,2.0,2.0,2.0,20.0,0.0,0,0,76,1173,0.000000,17,20131227.0
57,8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=,1,1.0,28.0,0.0,7.0,41.0,30.0,149.0,149.0,1.0,0.0,20.0,0.0,0.0,0.0,140.0,160.0,38807.650,0,10,2.000,0.0000,0.0000,0.0000,14.000,16.00,3880.0,0.000,0.0000,0.0000,0.0000,0.000,0.000,0.0,1,30.0,149.0,149.0,5,10,1,30,1,38807.6500,160.0,20.0,0.0,0.0,0.0,140.0,0.0,0,15,283,698,4.966667,479,20140109.0
67,K6fja4+jmoZ5xG6BypqX80Uw/XKpMgrEMdG2edFOxnA=,1,13.0,35.0,2.0,7.0,41.0,30.0,99.0,99.0,1.0,0.0,72.0,56.0,32.0,32.0,3928.0,4384.0,992830.000,24,96,0.600,0.4666,0.2666,0.2666,32.720,36.53,8270.0,0.712,0.8057,0.4421,0.4421,19.060,14.560,4612.0,15,30.0,99.0,99.0,7,8,29,16,0,66188.6640,292.0,5.0,4.0,2.0,2.0,262.0,0.0,0,0,171,1146,3.300000,15,20140125.0
187,ibIHVYBqxGwrSExE63/omeDD99M5vYB3CN2HzkEY+eM=,1,22.0,28.0,0.0,3.0,40.0,30.0,149.0,149.0,1.0,1.0,6.0,1.0,1.0,4.0,37.0,48.0,10110.665,2,2,1.500,0.2500,0.2500,1.0000,9.250,12.00,2528.0,1.658,0.4330,0.4330,0.0000,8.290,9.140,2036.0,4,30.0,149.0,149.0,1,1,18,27,0,2527.6663,12.0,2.0,0.0,0.0,1.0,9.0,0.0,0,13,0,1145,4.966667,15,20140126.0
191,kVmM8X4iBPCOfK/m1l3KR7LjFoxD1EcKYCc76F5IAWw=,1,1.0,28.0,0.0,7.0,41.0,30.0,99.0,99.0,1.0,1.0,2.0,0.0,0.0,0.0,14.0,16.0,3880.765,0,1,2.000,0.0000,0.0000,0.0000,14.000,16.00,3880.0,0.000,0.0000,0.0000,0.0000,0.000,0.000,0.0,1,30.0,99.0,99.0,1,1,1,30,1,3880.7650,16.0,2.0,0.0,0.0,0.0,14.0,0.0,0,15,0,1133,3.300000,24,20140129.0
192,moRTKhKIDvb+C8ZHOgmaF4dXMLk0jOn65d7a8tQ2Eds=,1,9.0,28.0,2.0,3.0,38.0,30.0,149.0,149.0,0.0,0.0,78.0,20.0,25.0,16.0,812.0,649.0,221420.770,8,23,2.516,0.6450,0.8066,0.5160,26.190,20.94,7144.0,2.723,0.7847,0.8200,0.7124,16.330,11.300,3914.0,31,30.0,149.0,149.0,1,1,31,0,0,7142.6050,21.0,3.0,1.0,1.0,1.0,26.0,0.0,0,0,408,1146,4.966667,7,20140202.0
223,dW/tPZMDh2Oz/ksduEctJbsz0MXw3kay/1AlZCq3EbI=,1,13.0,21.0,2.0,9.0,38.0,30.0,149.0,149.0,0.0,0.0,244.0,46.0,36.0,22.0,1186.0,1118.0,319777.620,14,44,4.207,0.7930,0.6206,0.3794,20.450,19.28,5510.0,4.285,0.9956,0.8870,0.6110,17.940,14.990,4760.0,29,30.0,149.0,149.0,2,2,31,2,0,11026.8140,39.0,8.0,2.0,1.0,1.0,41.0,0.0,0,0,98,1140,4.966667,3,20140212.0
281,otEcMhAX3mU4gumUSogqgteN3oaNmhdmTkoof2iRYEE=,1,13.0,28.0,0.0,7.0,40.0,30.0,149.0,149.

In [12]:
# getting shape of train dataset
wsdm_dataset.shape

(970960, 59)

In [13]:
# getting columns and converting them to list
cols = wsdm_dataset.columns.tolist()
# saving them for further use
np.savez_compressed(r"D:\Work_Masterarbeit\wsdm\preprocessed\columns.npz", cols)

In [14]:
# saving actual train dataset
wsdm_dataset.to_csv(r"D:\Work_Masterarbeit\wsdm\preprocessed\actual_train.csv", index=False)

In [ ]:
d

<h2> 3.2 Train Test Split

In [33]:
# loading actual wsdm data
wsdm_data = pd.read_csv(r"D:\Work_Masterarbeit\wsdm\preprocessed\actual_train.csv")
# setting max_columns to 100, so it displays all the columns of train data
pd.set_option('display.max_columns', 100)
# displaying head (top 5) rows of train data
wsdm_data.head()

,msno,is_churn,city,bd,gender,registered_via,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,is_cancel,num_25_sum,num_50_sum,num_75_sum,num_985_sum,num_100_sum,num_unq_sum,total_secs_sum,is_weekend_sum,is_weekday_sum,num_25_mean,num_50_mean,num_75_mean,num_985_mean,num_100_mean,num_unq_mean,total_secs_mean,num_25_std,num_50_std,num_75_std,num_985_std,num_100_std,num_unq_std,total_secs_std,active_days,payment_plan_days_mean,plan_list_price_mean,actual_amount_paid_mean,transaction_date_count,membership_expire_count,activity_period,inactive_days,is_rare,avg_time_perday,unq_track_perday,till_25_perday,till_50_perday,till_75_perday,till_985_perday,till_full_perday,discount,is_discount,days_since_last_login,days_left,layality_range,Perday_price,days_since_final_transaction,registration_init_time
0,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1,5.0,28.0,1.0,3.0,0.0,30.0,149.0,149.0,2.0,2.0,50.0,4.0,6.0,5.0,246.0,206.0,64499.977,5,6,4.547,0.3635,0.5454,0.4546,22.360,18.73,5864.0,4.734,0.4810,0.8906,0.8906,19.420,16.140,4748.0,11,30.0,149.0,149.0,1,1,29,20,0,5863.6343,19.0,5.0,0.0,1.0,0.0,22.0,0.0,0,2,19,1179,4.966667,15,20131223.0
1,f/NmvEzHfhINFEYZTR05prUdr+E+3+oewvweYz9cCQE=,1,13.0,20.0,1.0,3.0,36.0,30.0,180.0,180.0,0.0,0.0,0.0,4.0,2.0,0.0,26.0,30.0,6986.509,3,3,0.000,0.6665,0.3333,0.0000,4.332,5.00,1164.0,0.000,1.1050,0.4714,0.0000,2.926,2.887,741.5,6,30.0,180.0,180.0,1,1,20,25,0,1164.4182,5.0,0.0,1.0,0.0,0.0,4.0,0.0,0,11,11,1174,6.000000,20,20131223.0
2,zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,1,13.0,18.0,1.0,3.0,17.0,30.0,0.0,0.0,0.0,0.0,150.0,50.0,38.0,30.0,410.0,732.0,135620.940,16,24,3.750,1.2500,0.9500,0.7500,10.250,18.30,3390.0,2.980,1.4795,1.0710,0.7666,12.900,19.250,3882.0,20,30.0,74.5,74.5,2,2,31,11,0,6781.0470,37.0,8.0,2.0,2.0,2.0,20.0,0.0,0,0,76,1173,0.000000,17,20131227.0
3,8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=,1,1.0,28.0,0.0,7.0,41.0,30.0,149.0,149.0,1.0,0.0,20.0,0.0,0.0,0.0,140.0,160.0,38807.650,0,10,2.000,0.0000,0.0000,0.0000,14.000,16.00,3880.0,0.000,0.0000,0.0000,0.0000,0.000,0.000,0.0,1,30.0,149.0,149.0,5,10,1,30,1,38807.6500,160.0,20.0,0.0,0.0,0.0,140.0,0.0,0,15,283,698,4.966667,479,20140109.0
4,K6fja4+jmoZ5xG6BypqX80Uw/XKpMgrEMdG2edFOxnA=,1,13.0,35.0,2.0,7.0,41.0,30.0,99.0,99.0,1.0,0.0,72.0,56.0,32.0,32.0,3928.0,4384.0,992830.000,24,96,0.600,0.4666,0.2666,0.2666,32.720,36.53,8270.0,0.712,0.8057,0.4421,0.4421,19.060,14.560,4612.0,15,30.0,99.0,99.0,7,8,29,16,0,66188.6640,292.0,5.0,4.0,2.0,2.0,262.0,0.0,0,0,171,1146,3.300000,15,20140125.0


In [34]:
# Load RAW Data
wsdm_dataset_raw = pd.read_csv(r"D:\Work_Masterarbeit\wsdm\preprocessed_2\train.csv")

In [35]:
# Merging with RAW Data
wsdm_data = pd.merge(wsdm_dataset_raw, wsdm_data, on='msno', how='left')

In [36]:
## Sample 50% of Data due to memory restrictions
wsdm_data = wsdm_data.sample(frac=.5)

In [37]:
# replacing all nan and infinity to 0
wsdm_data = wsdm_data.replace(np.nan, 0, inplace=False)
wsdm_data = wsdm_data.replace(np.inf, 0, inplace=False)

In [39]:
#Doppelte Spalten entfernen
wsdm_data = wsdm_data.drop(['is_churn_y', 'city_y', 'bd_y', 'gender_y', 'registered_via_y', 'payment_method_id_y', 'payment_plan_days_y', 'plan_list_price_y', 'actual_amount_paid_y', 'is_auto_renew_y', 'is_cancel_y'], axis=1, inplace=False)

In [44]:
## Typkonvertierungen
wsdm_data['registration_init_time_x'] = wsdm_data['registration_init_time_x'].str.replace('-', '').astype(float)
wsdm_data['transaction_date'] = wsdm_data['transaction_date'].str.replace('-', '').astype(float)
wsdm_data['membership_expire_date'] = wsdm_data['membership_expire_date'].str.replace('-', '').astype(float)
wsdm_data['date'] = wsdm_data['date'].str.replace('-', '').astype(float)

In [45]:
# https://stackoverflow.com/a/29651514
def normalize(df):
    result1 = df.copy()
    for feature_name in df.columns:
        if (str(feature_name) != str('msno') and str(feature_name)!=str('is_churn_x')):
            max_value = df[feature_name].max()
            min_value = df[feature_name].min()
            result1[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result1


wsdm_dataset = normalize(wsdm_data)

In [46]:
# getting labels for y
labels = wsdm_dataset['is_churn_x'].values
labels

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [47]:
# getting labels for X
data = wsdm_dataset.drop(['msno', 'is_churn_x'], axis=1, inplace=False)
data

,city_x,bd_x,gender_x,registered_via_x,registration_init_time_x,payment_method_id_x,payment_plan_days_x,plan_list_price_x,actual_amount_paid_x,is_auto_renew_x,transaction_date,membership_expire_date,is_cancel_x,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs,num_25_sum,num_50_sum,num_75_sum,num_985_sum,num_100_sum,num_unq_sum,total_secs_sum,is_weekend_sum,is_weekday_sum,num_25_mean,num_50_mean,num_75_mean,num_985_mean,num_100_mean,num_unq_mean,total_secs_mean,num_25_std,num_50_std,num_75_std,num_985_std,num_100_std,num_unq_std,total_secs_std,active_days,payment_plan_days_mean,plan_list_price_mean,actual_amount_paid_mean,transaction_date_count,membership_expire_count,activity_period,inactive_days,is_rare,avg_time_perday,unq_track_perday,till_25_perday,till_50_perday,till_75_perday,till_985_perday,till_full_perday,discount,is_discount,days_since_last_login,days_left,layality_range,Perday_price,days_since_final_transaction,registration_init_time_y
5135652,0.590909,0.328571,1.0,0.230769,0.616451,0.878049,1.0,1.000000,1.000000,0.5,0.999357,0.003136,0.0,0.800000,0.266667,0.00,0.000000,0.000000,0.283784,0.358209,0.250025,0.006240,0.007135,0.003488,0.007993,0.006000,0.006529,0.006384,0.005769,0.007429,0.269000,0.232750,0.137933,0.333333,0.259054,0.353582,0.267989,0.471600,0.52400,0.328267,0.700667,0.329595,0.414328,0.337100,0.933333,1.0,1.000000,1.000000,0.000000,0.000000,1.000000,0.066667,0.0,0.005734,0.006797,0.004902,0.006329,0.000000,0.006944,0.005709,0.0,0.0,0.000000,0.007296,0.374739,1.000000,0.015873,0.616451
2162009,0.500000,0.428571,1.0,0.692308,0.159833,0.926829,1.0,0.827778,0.827778,0.0,0.999110,0.003070,0.0,0.300000,0.266667,0.25,0.333333,0.333333,0.364865,0.462687,0.378476,0.002080,0.002643,0.003198,0.003859,0.005061,0.004758,0.005330,0.004808,0.006137,0.108333,0.104175,0.152767,0.194500,0.264189,0.309701,0.270395,0.233600,0.28565,0.429333,0.506200,0.419459,0.468060,0.415991,0.766667,1.0,0.827778,0.827778,0.000000,0.000000,0.966667,0.233333,0.0,0.005784,0.005947,0.002451,0.000000,0.000000,0.006944,0.006010,0.0,0.0,0.000000,0.005579,0.793946,0.827778,0.021978,0.159833
1703607,1.000000,0.342857,0.5,0.692308,0.153569,0.951220,1.0,0.827778,0.827778,0.5,0.994909,0.003136,0.0,0.100000,0.000000,0.25,0.333333,0.000000,0.837838,0.044776,0.758477,0.004480,0.005285,0.003488,0.002205,0.012000,0.008721,0.012242,0.011538,0.010336,0.127267,0.113650,0.090900,0.060600,0.341486,0.309403,0.338701,0.333600,0.32800,0.358400,0.257133,0.412703,0.450149,0.391471,0.700000,1.0,0.827778,0.827778,0.020833,0.004831,1.000000,0.300000,0.0,0.014494,0.011895,0.004902,0.006329,0.006993,0.000000,0.015325,0.0,0.0,0.000000,0.020172,0.851983,0.827778,0.000000,0.153569
1439725,0.590909,0.385714,1.0,0.538462,0.698550,0.975610,1.0,0.827778,0.827778,0.5,0.999407,0.001485,0.5,0.166667,0.066667,0.00,0.333333,0.666667,0.189189,0.223881,0.202465,0.004800,0.011099,0.013372,0.013782,0.009087,0.012063,0.010032,0.015385,0.014858,0.096733,0.169300,0.247233,0.268867,0.183514,0.303433,0.196964,0.147067,0.44700,0.560867,0.596667,0.062081,0.459403,0.067164,1.000000,1.0,0.913889,0.913889,0.020833,0.004831,1.000000,0.000000,0.0,0.008430,0.011895,0.003676,0.006329,0.006993,0.013889,0.008113,0.0,0.0,0.000000,0.008155,0.258246,1.000000,0.013431,0.698550
12336930,0.681818,0.371429,0.5,0.692308,0.229735,0.829268,1.0,0.827778,0.827778,0.5,1.000000,0.003350,0.0,0.900000,0.066667,0.00,0.000000,0.000000,0.000000,0.000000,0.002325,0.004587,0.003171,0.004360,0.000276,0.002040,0.002228,0.002317,0.006731,0.005491,0.238933,0.125000,0.208333,0.013887,0.106419,0.137463,0.117465,0.471333,0.45650,0.571333,0.133200,0.245676,0.298030,0.247335,0.766667,1.0,0.827778,0.827778,0.000000,0.000000,1.000000,0.233333,0.0,0.002514,0.002549,0.004902,0.000000,0.006993,0.000000,0.002404,0.0,0.0,0.000000,0.012876,0.779749,0.827778,0.000000,0.229735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

<h3>Daten aufteilen in 70% Training und 30% Test

In [48]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.3, random_state=42)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(5910756, 68) (5910756,)
(2533182, 68) (2533182,)


In [49]:
X_train

,city_x,bd_x,gender_x,registered_via_x,registration_init_time_x,payment_method_id_x,payment_plan_days_x,plan_list_price_x,actual_amount_paid_x,is_auto_renew_x,transaction_date,membership_expire_date,is_cancel_x,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs,num_25_sum,num_50_sum,num_75_sum,num_985_sum,num_100_sum,num_unq_sum,total_secs_sum,is_weekend_sum,is_weekday_sum,num_25_mean,num_50_mean,num_75_mean,num_985_mean,num_100_mean,num_unq_mean,total_secs_mean,num_25_std,num_50_std,num_75_std,num_985_std,num_100_std,num_unq_std,total_secs_std,active_days,payment_plan_days_mean,plan_list_price_mean,actual_amount_paid_mean,transaction_date_count,membership_expire_count,activity_period,inactive_days,is_rare,avg_time_perday,unq_track_perday,till_25_perday,till_50_perday,till_75_perday,till_985_perday,till_full_perday,discount,is_discount,days_since_last_login,days_left,layality_range,Perday_price,days_since_final_transaction,registration_init_time_y
7493078,0.590909,0.314286,1.0,0.692308,0.690902,0.951220,1.000000,0.827778,0.827778,0.5,1.000000,0.005001,0.0,0.400000,0.000000,0.00,0.000000,0.000000,0.067568,0.000000,0.058766,0.014400,0.016913,0.022674,0.007718,0.014245,0.005324,0.016356,0.013462,0.014858,0.300000,0.266500,0.433333,0.155533,0.297297,0.130358,0.331797,0.588000,0.68200,0.772667,0.537133,0.494595,0.278507,0.551386,0.966667,1.000000,0.827778,0.827778,0.020833,0.004831,1.000000,0.033333,0.0,0.014201,0.005098,0.011029,0.012658,0.020979,0.006944,0.013221,0.0,0.0,0.000000,0.025751,0.322756,0.827778,0.000000,0.690902
15306368,0.045455,0.385714,0.0,0.538462,0.614106,1.000000,1.000000,0.550000,0.550000,0.5,0.999160,0.003086,0.0,0.100000,0.733333,0.00,0.000000,1.000000,0.135135,0.343284,0.161761,0.003467,0.003700,0.002907,0.003032,0.001943,0.002265,0.002163,0.003846,0.004199,0.254933,0.205925,0.196133,0.215667,0.143054,0.203731,0.154809,0.584400,0.52100,0.562667,0.557333,0.283351,0.357612,0.306183,0.533333,1.000000,0.550000,0.550000,0.000000,0.000000,1.000000,0.466667,0.0,0.003314,0.003965,0.004902,0.006329,0.006993,0.006944,0.003305,0.0,0.0,0.000000,0.006009,0.393737,0.550000,0.020757,0.614106
15480273,0.045455,0.385714,0.0,0.538462,0.697774,1.000000,1.000000,0.550000,0.550000,0.5,0.998616,0.002905,0.0,0.000000,0.000000,0.25,0.333333,0.000000,0.175676,0.089552,0.168595,0.001707,0.003964,0.004070,0.002481,0.004101,0.002612,0.004184,0.004808,0.005814,0.092800,0.163075,0.203000,0.130467,0.223243,0.171343,0.221231,0.254000,0.37915,0.471000,0.510400,0.223784,0.209761,0.228785,0.733333,1.000000,0.550000,0.550000,0.000000,0.000000,0.966667,0.266667,0.0,0.004739,0.003115,0.001225,0.006329,0.006993,0.000000,0.005108,0.0,0.0,0.033333,0.001288,0.261169,0.550000,0.034188,0.697774
1668848,1.000000,0.328571,1.0,0.230769,0.772425,0.951220,1.000000,0.827778,0.827778,0.5,1.000000,0.004968,0.0,0.266667,0.000000,0.25,0.000000,0.333333,0.081081,0.104478,0.099769,0.000960,0.007400,0.004651,0.014884,0.005158,0.005616,0.006373,0.005769,0.012274,0.027280,0.159000,0.121167,0.409333,0.146892,0.194030,0.176252,0.065560,0.35500,0.378667,0.664667,0.190676,0.269254,0.228358,0.700000,1.000000,0.827778,0.827778,0.020833,0.004831,0.966667,0.300000,0.0,0.007546,0.007647,0.001225,0.006329,0.006993,0.013889,0.006611,0.0,0.0,0.033333,0.024893,0.207516,0.827778,0.000000,0.772425
6221315,0.590909,0.400000,0.5,0.692308,0.766930,0.902439,1.000000,0.827778,0.827778,0.5,0.999654,0.003235,0.0,0.566667,0.066667,0.00,0.000000,0.000000,0.540541,0.343284,0.509253,0.004000,0.006871,0.004360,0.003583,0.010511,0.007524,0.010991,0.003846,0.005814,0.227200,0.295500,0.227200,0.197000,0.598378,0.544776,0.607948,0.320800,0.59600,0.647800,0.434000,0.534595,0.528358,0.548827,0.700000,1.000000,0.827778,0.827778,0.000000,0.000000,0.966667,0.300000,0.0,0.013014,0.010479,0.003676,0.006329,0.006993,0.006944,0.013221,0.0,0.0,0.000000,0.009871,0.253445,0.827778,0.008547,0.766930
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [50]:
# converting X_train to numpy array
wsdm_arr = X_train.values
wsdm_arr

array([[0.59090909, 0.31428571, 1.        , ..., 0.82777778, 0.        ,
        0.69090224],
       [0.04545455, 0.38571429, 0.        , ..., 0.55      , 0.02075702,
        0.61410629],
       [0.04545455, 0.38571429, 0.        , ..., 0.55      , 0.03418803,
        0.69777399],
       ...,
       [0.18181818, 0.58571429, 0.5       , ..., 0.        , 0.2967033 ,
        0.23129487],
       [0.04545455, 0.38571429, 0.        , ..., 0.55555556, 0.01953602,
        0.84630817],
       [0.18181818, 0.51428571, 1.        , ..., 0.82777778, 0.004884  ,
        0.69168627]])

In [51]:
# checking shape of train array
wsdm_arr.shape

(5910756, 68)

In [52]:
# saving it for further use
np.savez_compressed(r"D:\Work_Masterarbeit\wsdm\preprocessed\actual_train_incraw.npz", wsdm_arr)

In [53]:
# displaying y_train
wsdm_labels_arr = y_train
wsdm_labels_arr

array([0, 0, 0, ..., 1, 0, 0], dtype=int64)

In [54]:
# saving it for further use
np.savez_compressed(r"D:\Work_Masterarbeit\wsdm\preprocessed\actual_wsdm_labels_incraw.npz", wsdm_labels_arr)

In [55]:
# converting X_test to numpy array
cv_arr = X_test.values
cv_arr

array([[0.22727273, 0.52857143, 0.5       , ..., 0.82777778, 0.        ,
        0.53874003],
       [0.04545455, 0.38571429, 0.        , ..., 0.55      , 0.02930403,
        0.85151962],
       [0.04545455, 0.38571429, 0.        , ..., 0.55      , 0.01953602,
        0.54410521],
       ...,
       [0.59090909, 0.48571429, 0.5       , ..., 0.82777778, 0.01098901,
        0.08226875],
       [0.45454545, 0.34285714, 0.5       , ..., 0.82777778, 0.        ,
        0.38508663],
       [0.04545455, 0.38571429, 0.        , ..., 0.82777778, 0.00854701,
        0.69171701]])

In [56]:
# checking shape of cv array
cv_arr.shape

(2533182, 68)

In [57]:
# saving it for further use
np.savez_compressed(r"D:\Work_Masterarbeit\wsdm\preprocessed\actual_cv_incraw.npz", cv_arr)

In [58]:
# displaying y_test
cv_labels_arr = y_test
cv_labels_arr

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [59]:
# checking shape of cv labels array
cv_labels_arr.shape

(2533182,)

In [60]:
# saving it for further use
np.savez_compressed(r"D:\Work_Masterarbeit\wsdm\preprocessed\actual_cv_labels_incraw.npz", cv_labels_arr)

<h1> 5. Modelling

In [17]:
# loading library
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import joblib
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import log_loss

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb


In [18]:
# reference: https://machinelearningmastery.com/how-to-save-a-numpy-array-to-file-for-machine-learning/

X_train = np.load(r"D:\Work_Masterarbeit\wsdm\preprocessed\actual_train_incraw.npz")
y_train = np.load(r"D:\Work_Masterarbeit\wsdm\preprocessed\actual_wsdm_labels_incraw.npz")
X_test = np.load(r"D:\Work_Masterarbeit\wsdm\preprocessed\actual_cv_incraw.npz")
y_test = np.load(r"D:\Work_Masterarbeit\wsdm\preprocessed\actual_cv_labels_incraw.npz")


In [19]:
X_train = X_train['arr_0']
X_test = X_test['arr_0']
y_train = y_train['arr_0']
y_test = y_test['arr_0']

# getting shapes
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(5910756, 68) (5910756,)
(2533182, 68) (2533182,)


In [20]:
# loading columns
cols = np.load(r"D:\Work_Masterarbeit\wsdm\preprocessed\columns.npz")
# converting numpy array to python list
cols = cols['arr_0'].tolist()
# removing first two cols 'msno' and 'is_churn'
cols = cols[2:]

<h2> Logistic Regression

In [21]:
%%time

# fitting logistic regression model with best parameter values
lr_model = LogisticRegression(penalty='l2', tol=0.0001, C=10, solver='newton-cg', class_weight='balanced', n_jobs=-1)
lr_model.fit(X_train, y_train)

Wall time: 2h 31min 49s


LogisticRegression(C=10, class_weight='balanced', n_jobs=-1, solver='newton-cg')

In [22]:
# Reference: https://machinelearningmastery.com/save-load-machine-learning-models-python-scikit-learn/#:~:text=Saving%20Your%20Model-,Save%20Your%20Model%20with%20pickle,it%20to%20make%20new%20predictions.
# saving the trained logistic regression ML model
filename = r"D:\Work_Masterarbeit\wsdm\preprocessed\finalized_model_lr.sav"
joblib.dump(lr_model, filename)

['D:\\Work_Masterarbeit\\wsdm\\preprocessed\\finalized_model_lr.sav']

In [23]:
# loading the best model from disk
filename = r"D:\Work_Masterarbeit\wsdm\preprocessed\finalized_model_lr.sav"
loaded_model = joblib.load(filename)

<h3> Model Evaluation

In [24]:
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score

lr_predict=lr_model.predict(X_test)
lr_predict


print('Accuracy:', accuracy_score(y_test, lr_predict))
print('F1 score weighted:', f1_score(y_test, lr_predict,average='weighted'))
print('F1 score macro:', f1_score(y_test, lr_predict, average='macro'))
print('Recall:', recall_score(y_test, lr_predict,average='weighted'))
print('Precision:', precision_score(y_test, lr_predict,average='weighted'))
print('\n clasification report:\n', classification_report(y_test, lr_predict))
print('\n confussion matrix:\n',confusion_matrix(y_test, lr_predict))

Accuracy: 0.84941232015702
F1 score weighted: 0.8692327045953105
F1 score macro: 0.706532662601837
Recall: 0.84941232015702
Precision: 0.9061671189731874

 clasification report:
               precision    recall  f1-score   support

           0       0.97      0.86      0.91   2272964
           1       0.38      0.74      0.50    260218

    accuracy                           0.85   2533182
   macro avg       0.67      0.80      0.71   2533182
weighted avg       0.91      0.85      0.87   2533182


 confussion matrix:
 [[1959633  313331]
 [  68135  192083]]


In [25]:
## Logloss
# predicting probabilities for X_cv
pred_test = loaded_model.predict_proba(X_test)
# getting probabilities corresponding to class label 1 only
pred_test = pred_test[:,1]
# calculating log loss
print('Test log-loss: ', log_loss(y_test, pred_test))

Test log-loss:  0.4503247950138433


<h2> Decision Tree

In [26]:
%%time

# training a decision tree classifier model with best parameters
dtc_model = DecisionTreeClassifier(criterion='gini', splitter='best', max_depth=5, min_samples_split=3, min_samples_leaf=1)
dtc_model.fit(X_train, y_train)

Wall time: 2min 19s


DecisionTreeClassifier(max_depth=5, min_samples_split=3)

In [27]:
# https://machinelearningmastery.com/save-load-machine-learning-models-python-scikit-learn/#:~:text=Saving%20Your%20Model-,Save%20Your%20Model%20with%20pickle,it%20to%20make%20new%20predictions.
# saving the decision tree classifier model
filename = r"D:\Work_Masterarbeit\wsdm\preprocessed\finalized_model_dtc.sav"
joblib.dump(dtc_model, filename)

['D:\\Work_Masterarbeit\\wsdm\\preprocessed\\finalized_model_dtc.sav']

In [28]:
# loading the best decision tree classifier model from disk
filename = r"D:\Work_Masterarbeit\wsdm\preprocessed\finalized_model_dtc.sav"
loaded_model = joblib.load(filename)

<h3> Model Evaluation

In [29]:
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score

dtc_modelpred=dtc_model.predict(X_test)
dtc_modelpred


print('Accuracy:', accuracy_score(y_test, dtc_modelpred))
print('F1 score weighted:', f1_score(y_test, dtc_modelpred,average='weighted'))
print('F1 score macro:', f1_score(y_test, dtc_modelpred, average='macro'))
print('Recall:', recall_score(y_test, dtc_modelpred,average='weighted'))
print('Precision:', precision_score(y_test, dtc_modelpred,average='weighted'))
print('\n clasification report:\n', classification_report(y_test, dtc_modelpred))
print('\n confussion matrix:\n',confusion_matrix(y_test, dtc_modelpred))

Accuracy: 0.9626414525288748
F1 score weighted: 0.9605623571366396
F1 score macro: 0.8875489409671232
Recall: 0.9626414525288748
Precision: 0.9612982746144014

 clasification report:
               precision    recall  f1-score   support

           0       0.97      0.99      0.98   2272964
           1       0.91      0.71      0.80    260218

    accuracy                           0.96   2533182
   macro avg       0.94      0.85      0.89   2533182
weighted avg       0.96      0.96      0.96   2533182


 confussion matrix:
 [[2254303   18661]
 [  75975  184243]]


In [30]:
## Logloss
# predicting probabilities for X_cv
pred_test = loaded_model.predict_proba(X_test)
# getting probabilities corresponding to class label 1 only
pred_test = pred_test[:,1]
# calculating log loss
print('Test log-loss: ', log_loss(y_test, pred_test))

Test log-loss:  0.12499157891397941


<h2> Random Forest

In [31]:
%%time

# training the random forest classifier for best hyper parameters
rf_model = RandomForestClassifier(n_estimators=500, criterion='gini', max_depth=15, min_samples_split=3, min_samples_leaf=2, max_features='sqrt', bootstrap=False, n_jobs=-1)
rf_model.fit(X_train, y_train)

Wall time: 56min 42s


RandomForestClassifier(bootstrap=False, max_depth=15, max_features='sqrt',
                       min_samples_leaf=2, min_samples_split=3,
                       n_estimators=500, n_jobs=-1)

In [32]:
# https://machinelearningmastery.com/save-load-machine-learning-models-python-scikit-learn/#:~:text=Saving%20Your%20Model-,Save%20Your%20Model%20with%20pickle,it%20to%20make%20new%20predictions.
# saving the decision tree classifier model
filename = r"D:\Work_Masterarbeit\wsdm\preprocessed\finalized_model_rf.sav"
joblib.dump(rf_model, filename)

['D:\\Work_Masterarbeit\\wsdm\\preprocessed\\finalized_model_rf.sav']

In [33]:
# loading the best model from disk
filename = r"D:\Work_Masterarbeit\wsdm\preprocessed\finalized_model_rf.sav"
loaded_model = joblib.load(filename)

<h3>Model Evaluation

In [34]:
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score

rf_modelpred=rf_model.predict(X_test)
rf_modelpred


print('Accuracy:', accuracy_score(y_test, rf_modelpred))
print('F1 score weighted:', f1_score(y_test, rf_modelpred,average='weighted'))
print('F1 score macro:', f1_score(y_test, rf_modelpred, average='macro'))
print('Recall:', recall_score(y_test, rf_modelpred,average='weighted'))
print('Precision:', precision_score(y_test, rf_modelpred,average='weighted'))
print('\n clasification report:\n', classification_report(y_test, rf_modelpred))
print('\n confussion matrix:\n',confusion_matrix(y_test, rf_modelpred))

Accuracy: 0.9781764594885011
F1 score weighted: 0.9774932381728749
F1 score macro: 0.9371319053637046
Recall: 0.9781764594885011
Precision: 0.9778061418676611

 clasification report:
               precision    recall  f1-score   support

           0       0.98      1.00      0.99   2272964
           1       0.95      0.83      0.89    260218

    accuracy                           0.98   2533182
   macro avg       0.97      0.91      0.94   2533182
weighted avg       0.98      0.98      0.98   2533182


 confussion matrix:
 [[2262358   10606]
 [  44677  215541]]


In [35]:
## Logloss
# predicting probabilities for X_cv
pred_test = loaded_model.predict_proba(X_test)
# getting probabilities corresponding to class label 1 only
pred_test = pred_test[:,1]
# calculating log loss
print('Test log-loss: ', log_loss(y_test, pred_test))

Test log-loss:  0.07116937326829802
